In [7]:
import liblibra_core
from libra_py import *
import math
import py3Dmol

In [8]:
params = {"nsteps":100, "nperiods":1, "scale":250.0, "visualize":0, "verbosity":0, "prefix":"_Si8_cov_flag0"}

R, V, A, M, E = QE_methods.read_md_data("x0.xml")

### Method 1:

Strachan, A. Normal Modes and Frequencies from Covariances in Molecular Dynamics 
    or Monte Carlo Simulation. J. Chem. Phys. 2003, 120, 1-4.
    
In the following 2 calls *res* is a tuple (w, w_inv_cm, U_v,  w2, w2_inv_cm, U_a)

In [9]:
params.update({"cov_flag":0})
res00 = normal_modes.compute_cov( R, V, A, M, E, params)

Another option is to use data centered around the averages (fluctuations)

In [10]:
params.update({"cov_flag":1})
res01 = normal_modes.compute_cov( R, V, A, M, E, params)


### Method 2:
Pereverzev, A.; Sewell, T. D. Obtaining the Hessian from the Force Covariance Matrix:
    Application to Crystalline Explosives PETN and RDX. J. Chem. Phys. 2015, 142, 134110.
    
In the following 2 calls *res* is a tuple (w_a, w_inv_cm, U_a)
Again, without centering:

In [11]:
params.update({"cov_flag":0})
res20 = normal_modes.compute_cov2( R, A, M, E, 300.0, params)

With the centering:

In [12]:
params.update({"cov_flag":1})
res21 = normal_modes.compute_cov2( R, A, M, E, 300.0, params)

Now, we can select which normal modes to visualize:

In [17]:
mode = 10

# Method 1
xyz00v = normal_modes.get_xyz(E,R, M, res00[2], mode)  # velocity covariance
xyz00a = normal_modes.get_xyz(E,R, M, res00[5], mode)  # acceleration covariance
xyz01v = normal_modes.get_xyz(E,R, M, res01[2], mode)  # velocity fluctuation covariance
xyz01a = normal_modes.get_xyz(E,R, M, res01[5], mode)  # acceleration fluctuation covariance

# Method 2
xyz20f = normal_modes.get_xyz(E,R, M, res20[2], mode)  # force covariance
xyz21f = normal_modes.get_xyz(E,R, M, res21[2], mode)  # force fluctuation covariance


view = py3Dmol.view(width=800,height=400, linked=False,viewergrid=(3,2))
view.addModel(xyz00v,'xyz',{'vibrate': {'frames':10,'amplitude':500}}, viewer=(0,0))
view.addModel(xyz01v,'xyz',{'vibrate': {'frames':10,'amplitude':500}}, viewer=(0,1))

view.addModel(xyz00a,'xyz',{'vibrate': {'frames':10,'amplitude':500}}, viewer=(1,0))
view.addModel(xyz01a,'xyz',{'vibrate': {'frames':10,'amplitude':500}}, viewer=(1,1))

view.addModel(xyz20f,'xyz',{'vibrate': {'frames':10,'amplitude':500}}, viewer=(2,0))
view.addModel(xyz21f,'xyz',{'vibrate': {'frames':10,'amplitude':500}}, viewer=(2,1))

view.setBackgroundColor('0xeeeeee')
view.setStyle({'sphere':{}})
view.animate({'loop': 'backAndForth'})
view.zoomTo()
view.show()

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol